In [0]:
# 데이터는 100배수만을 이용. (0, 100, 200, ...번째 데이터) 
from sklearn.datasets import fetch_california_housing
import pandas as pd

data, target = fetch_california_housing(as_frame=True, return_X_y=True)

for index, row in data.iterrows():
    if(index % 100 != 0):
        data = data.drop(index=[index])
# print(data)

target = target.to_frame()
for index, row in target.iterrows():
    if(index % 100 != 0):
        target = target.drop(index=[index])
# print(target)

In [0]:
# 1. SparkSQL을 이용하여 수행하라. target의 평균값을 출력하라. 

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.createDataFrame(target)
# df.show()
df.select(avg("MedHouseVal")).show()

+------------------+
|  avg(MedHouseVal)|
+------------------+
|2.0160438164251206|
+------------------+



In [0]:
# 2.(2) 입력변수 중 임의개수를 활용하여 5개로 클러스터링하라. (클러스터링 알고리즘은 자유이나 Spark 이용) 그리고, 각 클러스터의 centroid를 출력하라. 
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

dataset = spark.createDataFrame(data)
# dataset.show()

# 5개
housing_df = dataset.select('MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population')
featureCols = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population']
assembler = VectorAssembler(inputCols=featureCols, outputCol='features')
assembled_df = assembler.transform(dataset)
# assembled_df.show(10, truncate=True)

kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(assembled_df)

# Make predictions
predictions = model.transform(assembled_df)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[3.79682609e+00 3.08550725e+01 5.12816717e+00 1.05730848e+00
 1.24260870e+03]
[6.06660000e+00 1.00000000e+01 6.52651839e+00 9.73053892e-01
 7.26600000e+03]
[4.48612500e+00 2.23333333e+01 5.31802195e+00 1.05475252e+00
 3.55308333e+03]
[3.27541364e+00 2.38863636e+01 5.08001723e+00 1.09532096e+00
 2.02552273e+03]
[  3.82294568  30.16049383   5.56381394   1.10849192 637.95061728]


In [0]:
# 3. (3점) 2번에서 선택해서 푼 결과를 이용하여, 전체 데이터에 대하여 실행하여 결과를 출력하는 프로그램을 작성하시오. 

from sklearn.datasets import fetch_california_housing
import pandas as pd
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

data_all, target_all = fetch_california_housing(as_frame=True, return_X_y=True)

dataset = spark.createDataFrame(data_all)
# dataset.show()

# 5개
housing_df = dataset.select('MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population')
featureCols = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population']
assembler = VectorAssembler(inputCols=featureCols, outputCol='features')
assembled_df = assembler.transform(dataset)
# assembled_df.show(10, truncate=True)

kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(assembled_df)

# Make predictions
predictions = model.transform(assembled_df)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[  3.93843525  31.78187609   5.68724719   1.13220554 724.23158327]
[3.96513270e+00 1.74881517e+01 5.25773486e+00 1.07317762e+00
 4.63796051e+03]
[3.77512292e+00 2.23812000e+01 5.07518521e+00 1.06692280e+00
 2.67109360e+03]
[3.79580120e+00 2.78984559e+01 5.22424049e+00 1.06346436e+00
 1.51498260e+03]
[4.56522803e+00 1.18409091e+01 5.76380922e+00 1.07572319e+00
 8.95328788e+03]
